# Library

In [4]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
tqdm.pandas()
# from CHAID import Tree
import ast

import optbinning
from optbinning import BinningProcess,ContinuousOptimalBinning
from optbinning import OptimalBinning

from datetime import datetime
from dateutil.relativedelta import relativedelta

# import pyreadstat


# train-test split

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Folder Paths

In [5]:
# path_main=r"D:\Pranto\Working\LSA and Writeoff\Data\Updated\Final Updated V1"
# month=r"JUN25""

path_main=r"C:\Users\866105\Downloads"
path_binning=path_main

# path_main=r"C:\Users\866105\OneDrive - Tata Capital\Pranto\WORK\Application Scorecard\INTRA\V1\DATA"

# path_binning=r"C:\Users\866105\OneDrive - Tata Capital\Pranto\WORK\Application Scorecard\INTRA\V1\BINNING FILE\V2"


names=rf"UV_Repurchase_Jan_Oct25 1"
# binning_file_name=rf"Binning_of_rep_Final"
binning_file_name=rf"Repurchase Raw Data_biining file with nearest bad rate"

# names=rf"INTRA RAW DATA V4"

# binning_file_name=rf"Segment_INTRA Raw Binning File V1"

# Numerical binning

### Numerical binning Data

In [6]:

# main Data

try:
    df_ftu=pd.read_csv(rf"{path_main}\{names}.csv",encoding='latin-1')

except:
    df_ftu=pd.read_excel(rf"{path_main}\{names}.xlsx")




# # Numwrical Binning File
# df_num_binning=pd.read_excel(rf"{path_binning}\{binning_file_name}.xlsx",sheet_name="Numerical Binning")
# # df_num_binning=pd.read_excel(rf"{path_binning}\{binning_file_name}.xlsx",sheet_name="Numerical Binning Data")

# print("Main Dataset Shape",df_ftu.shape)
# print("Binning Dataset Shape",df_num_binning.shape)


# print(df_num_binning.columns)


# df_ftu.head(3)

In [7]:
# Numwrical Binning File
df_num_binning=pd.read_excel(rf"{path_binning}\{binning_file_name}.xlsx",sheet_name="Numerical Binning")
# df_num_binning=pd.read_excel(rf"{path_binning}\{binning_file_name}.xlsx",sheet_name="Numerical Binning Data V1")

print("Main Dataset Shape",df_ftu.shape)
print("Binning Dataset Shape",df_num_binning.shape)


print(df_num_binning.columns)


df_ftu.head(3)

Main Dataset Shape (7768, 234)
Binning Dataset Shape (41, 13)
Index(['Variable', 'Group', 'Bin', 'Count', 'Count (%)', 'Non-event', 'Event',
       'Event rate', 'WoE', 'IV', 'JS', 'Method', 'Monotonic Trend'],
      dtype='object')


d:\Pranto\Working\Software\Python\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Pranto\Working\Software\Python\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,Ref ID,CONNO,LOGIN_DATE,REQUEST-TIME,AD_B,AGE_B,AGE_NEW_V1,APPLICATION_USAGE,ASSET_AGE_B,ASSET_AGE_NEW,...,Utilisation_Sec,Max_age_of_trade_Open,Avg_age_of_trade_Open,Min_age_of_trade_Secured,Avg_age_of_trade_Unsecured,Months_since_last_enq_CD,Months_since_last_enq_PL,Sum_of_Sancamt_Closed,amt_tenure_opendt,UV_REP_SEGMENT_V1
0,R30325936177,8000321810,30102025,30-10-2025 21:25,0.000000,25,25,Agriculture & Related Item,32.04,32.04,...,0.823829,21.612903,13.774194,13.064516,19.668622,30.690789,20.657895,2.528780e+05,2.947303e+07,UV_REP_SCV
1,R30225935761,8000321436,29102025,29-10-2025 22:34,14.090900,31,31,Agriculture & Related Item,63.00,63.00,...,NaN,NaN,NaN,NaN,6.096774,27.335526,NaN,1.199900e+04,3.088775e+05,UV_REP_SCV
2,R30225935578,8000321683,29102025,29-10-2025 21:49,23.855856,51,51,Bus Operation - Staff Transportation,57.96,57.96,...,1.022714,9.387097,5.865591,4.849462,24.559140,6.496711,18.338816,1.218126e+06,1.208510e+08,UV_REP_ICV


In [8]:
data=df_ftu
binning_df=df_num_binning

# Change the inputs according to the binning file

In [9]:
name = 'Bin'  # Columns name

# /////////////////////////////////////////////////////////////////////////////////////////
# Columns name according to Binning file(categorical,numerical)

Variable = 'Variable'  # Variable name
Bin = 'Bin'  # Bins
Group = 'Group'  # Group/WoE Value
Bad_Rate = 'Event rate'

# /////////////////////////////////////////////////////////////////////////////////////////
# Target Variable name for checking nearest bad rate

target = None  # Set target to None to test dynamic handling
# target = 'TARGET'  # Uncomment to provide a target


# /////////////////////////////////////////////////////////////////////////////////////////

# Specify the method to use for unmapped values: "lowest", "highest", "average", or "nearest"
methods = "highest"  # Set the desired method here

# Apply binning for numerical columns

In [10]:
# # Clean binning file data_V1
# def clean_data_V1frame(df):
#     try:
#         df = df[df['Group'] != 'Totals']
#         # Convert Group to numeric for manipulation (Totals becomes NaN)
#         df["Group"] = pd.to_numeric(df["Group"], errors="coerce")

#         # Drop rows where Group is "Totals"
#         df = df.dropna(subset=["Group"])

#         # Drop rows where Bin is "Special" or "(-inf, inf)"
#         df = df[~df["Bin"].isin(["Special", "(-inf, inf)"])]

#         # Process "Missing" values
#         mask_missing = (df["Bin"] == "Missing") & (df["Count"] > 0)
#         df.loc[mask_missing, "Group"] = df.loc[mask_missing, "Group"] - 1  # Adjust Group

#         # Drop other "Missing" values where Count is 0
#         df = df[~((df["Bin"] == "Missing") & (df["Count"] == 0))]

#         # Reset index
#         df = df.reset_index(drop=True)

#         return df
#     except:
#         pass

# # Clean data_V1
# binning_df=clean_data_V1frame(binning_df)



# Function to convert bin strings to tuples
def parse_bin(bin_str):
    if bin_str == 'Missing':
        return None
    bounds = bin_str.strip('()[]').split(',')
    return (float(bounds[0]), float(bounds[1]))

# Group by Variable and create dictionary of bin ranges with corresponding group values
bin_ranges = {}
bin_groups = {}
for variable, group_data in binning_df.groupby(Variable):
    bin_ranges[variable] = [parse_bin(bin_str) for bin_str in group_data[Bin]]
    bin_groups[variable] = group_data[Group].tolist()

# Function to map values to bins and retrieve corresponding group value
def map_to_bins(value, bins, groups):
    if pd.isna(value):
        return groups[-1]  # If value is missing, return the last group value
    for i, bin_range in enumerate(bins):
        if bin_range is not None and bin_range[0] <= value < bin_range[1]:
            return groups[i]
    # return groups[-1]  # If value not found in any bin, return the last group value
    return None

# Apply binning and create new column_B with the last group value for missing values
for column in binning_df[Variable].unique():
    if f'{column}_{name}' not in data:  # Check if the column_B does not exist in the dataset
        bins = bin_ranges[column]
        groups = bin_groups[column]
        data[f'{column}_{name}'] = [map_to_bins(value, bins, groups) for value in data[column]]



# try:

#     # Apply binning and create new column_B with the last group value for missing values
#     for column in binning_df[Variable].unique():
#         if f'{column}_{name}' not in data:  # Check if the column_B does not exist in the dataset
#             bins = bin_ranges[column]
#             groups = bin_groups[column]
#             data[f'{column}_{name}'] = [map_to_bins(value, bins, groups) for value in data[column]]

# except:
    
#     pass

# Create DataFrame with new column_B
df_binned = pd.DataFrame(data)

print(df_binned.shape)

df_binned.head(3)


(7768, 245)


,Ref ID,CONNO,LOGIN_DATE,REQUEST-TIME,AD_B,AGE_B,AGE_NEW_V1,APPLICATION_USAGE,ASSET_AGE_B,ASSET_AGE_NEW,...,Peak_Delay_cibil_1yrs_Bin,ASSET_AGE_NEW_Bin,Total_Loans_Bin,Secured_Loans_Bin,Total_Enquiry_count_12_month_Bin,Sum_of_Sancamt_Closed_Bin,Total_enquiries_L6M_Bin,Max_age_of_trade_Bin,Months_since_last_enq_PL_Bin,Months_since_last_enq_CD_Bin
0,R30325936177,8000321810,30102025,30-10-2025 21:25,0.000000,25,25,Agriculture & Related Item,32.04,32.04,...,1,0,2,1,0,1,0,1,1,1
1,R30225935761,8000321436,29102025,29-10-2025 22:34,14.090900,31,31,Agriculture & Related Item,63.00,63.00,...,2,1,0,0,0,0,1,0,2,1
2,R30225935578,8000321683,29102025,29-10-2025 21:49,23.855856,51,51,Bus Operation - Staff Transportation,57.96,57.96,...,0,1,2,2,0,1,0,1,1,1


In [11]:
df_binned.columns

Index(['Ref ID', 'CONNO', 'LOGIN_DATE', 'REQUEST-TIME', 'AD_B', 'AGE_B',
       'AGE_NEW_V1', 'APPLICATION_USAGE', 'ASSET_AGE_B', 'ASSET_AGE_NEW',
       ...
       'Peak_Delay_cibil_1yrs_Bin', 'ASSET_AGE_NEW_Bin', 'Total_Loans_Bin',
       'Secured_Loans_Bin', 'Total_Enquiry_count_12_month_Bin',
       'Sum_of_Sancamt_Closed_Bin', 'Total_enquiries_L6M_Bin',
       'Max_age_of_trade_Bin', 'Months_since_last_enq_PL_Bin',
       'Months_since_last_enq_CD_Bin'],
      dtype='object', length=245)

In [12]:
df_binned.columns.to_list()

['Ref ID',
 'CONNO',
 'LOGIN_DATE',
 'REQUEST-TIME',
 'AD_B',
 'AGE_B',
 'AGE_NEW_V1',
 'APPLICATION_USAGE',
 'ASSET_AGE_B',
 'ASSET_AGE_NEW',
 'UV_REP_SEGMENT',
 'AUTO_LOAN_COMPLETED',
 'AUTO_LOAN_LIVE',
 'BOW_CV_BIZ_EXP_B',
 'BRANCH_NEW',
 'CHASSIS_FULLY_BUILT',
 'CO_APP_RELN_BOW',
 'COBORROWER_YN_node',
 'COMMERCIAL_VEHI_COMPLETED',
 'CUST_PROFILE_ASSET_CATEGORY',
 'CUST_SEG_NODE',
 'DIST_SALES_OUTLET',
 'DL_CLASS',
 'DL_YES_NO',
 'DSA_DLR',
 'EXISTING_CUST_YN_NODE',
 'EXISTING_FINANCIER',
 'FIN_AMOUNT',
 'FTF_REPEAT_MULTIPLE',
 'GENDER_NODE',
 'HOUSING_LOAN_COMPLETED',
 'HOUSING_LOAN_LIVE',
 'INCOME',
 'IT_RETURNS',
 'IT_RETURNS_YES_NO',
 'LTV',
 'MARITAL_STATUS',
 'NUM_VEHICLES',
 'Num_Vehicle_New_Node',
 'PRODUCT',
 'PROPERT_TYPE',
 'PROPERT_TYPE_New',
 'PROPERT_TYPE_YES_NO',
 'CUST_DL_PROP_ITR_V1',
 'CUST_DL_PROP',
 'RESI_OWNERSHIP',
 'Resi_Ownership_New_Node',
 'SCHEME_CODE',
 'SOURCING',
 'TENURE_B',
 'UNSECURED_LOANS_YN',
 'VEHI_COST',
 'INDUCTIVE_EMI',
 'NET_ABB',
 'MAX_ACB_

In [13]:
a="Avg_Delay_cibil_Bin"


df_binned[a].value_counts()

Avg_Delay_cibil_Bin
0    4403
3    1585
1    1464
2     316
Name: count, dtype: int64

In [14]:
# df_binned.to_csv("abc_num_bin.csv",index=False)

# Categorical Binning

### Categorical binning dataframe

In [15]:
df_ftu=df_ftu

# Categorical binning File
# df_cat_bin=pd.read_excel(rf"{path_binning}\{binning_file_name}.xlsx",sheet_name="Categorical Binning Data V1")

df_cat_bin=pd.read_excel(rf"{path_binning}\{binning_file_name}.xlsx",sheet_name="Categorical Binning")

print("Main Dataset Shape",df_ftu.shape)
print("Binning Dataset Shape",df_cat_bin.shape)


print(df_cat_bin.columns)


df_ftu.head(3)


Main Dataset Shape (7768, 245)
Binning Dataset Shape (16, 14)
Index(['Variable', 'Group', 'Bin', 'Count', 'Count (%)', 'Non-event', 'Event',
       'Event rate', 'WoE', 'IV', 'JS', 'Method', 'Monotonic Trend',
       'Length of Bins'],
      dtype='object')


,Ref ID,CONNO,LOGIN_DATE,REQUEST-TIME,AD_B,AGE_B,AGE_NEW_V1,APPLICATION_USAGE,ASSET_AGE_B,ASSET_AGE_NEW,...,Peak_Delay_cibil_1yrs_Bin,ASSET_AGE_NEW_Bin,Total_Loans_Bin,Secured_Loans_Bin,Total_Enquiry_count_12_month_Bin,Sum_of_Sancamt_Closed_Bin,Total_enquiries_L6M_Bin,Max_age_of_trade_Bin,Months_since_last_enq_PL_Bin,Months_since_last_enq_CD_Bin
0,R30325936177,8000321810,30102025,30-10-2025 21:25,0.000000,25,25,Agriculture & Related Item,32.04,32.04,...,1,0,2,1,0,1,0,1,1,1
1,R30225935761,8000321436,29102025,29-10-2025 22:34,14.090900,31,31,Agriculture & Related Item,63.00,63.00,...,2,1,0,0,0,0,1,0,2,1
2,R30225935578,8000321683,29102025,29-10-2025 21:49,23.855856,51,51,Bus Operation - Staff Transportation,57.96,57.96,...,0,1,2,2,0,1,0,1,1,1


In [16]:

data=df_ftu

binning_df=df_cat_bin

# Categorical Binning Apply

In [17]:

# # Clean binning file data_V1
# def clean_data_V1frame(df):
#     try:
#         df = df[df['Group'] != 'Totals']
#         # Convert Group to numeric for manipulation (Totals becomes NaN)
#         df["Group"] = pd.to_numeric(df["Group"], errors="coerce")

#         # Drop rows where Group is "Totals"
#         df = df.dropna(subset=["Group"])

#         # Drop rows where Bin is "Special" or "(-inf, inf)"
#         df = df[~df["Bin"].isin(["Special", "(-inf, inf)"])]

#         # Process "Missing" values
#         mask_missing = (df["Bin"] == "Missing") & (df["Count"] > 0)
#         df.loc[mask_missing, "Group"] = df.loc[mask_missing, "Group"] - 1  # Adjust Group

#         # Drop other "Missing" values where Count is 0
#         df = df[~((df["Bin"] == "Missing") & (df["Count"] == 0))]

#         # Reset index
#         df = df.reset_index(drop=True)

#         return df
#     except:
#         pass

# # Clean data_V1
# binning_df=clean_data_V1frame(binning_df)



# Function to map values to bins and retrieve corresponding group value
def map_to_bins_categorical(value, bins, groups):
    if pd.isna(value):
        return groups[-1] ##if groups else None##  # Handle missing group gracefully
    for i, bin_str in enumerate(bins):
        bin_values = [s for s in bin_str.strip("[]").split("'") if s]
        bin_values1 = [int(x) if x.isdigit() else x for x in bin_values]
        if isinstance(value, str):
            if value in bin_values:
                return groups[i]
        else:
            if value in bin_values1:
                return groups[i]
    return None  # If value not found in any bin, return None

# ////////////////////////////////////////////////////////////////////////////////////////////////////////////

# Function to compute bad rate for each category in a column
def compute_bad_rate(df, column, target):
    bad_rate_df = df.groupby(column)[target].mean().reset_index()
    bad_rate_df.columns = [column, Bad_Rate]  # Rename column for consistency
    return bad_rate_df

# ////////////////////////////////////////////////////////////////////////////////////////////////////////////

def find_nearest_group(bad_rate, binning_df, variable, method):
    variable_binning = binning_df[binning_df[Variable] == variable]
    if method == "nearest":
        # Find the nearest bad rate
        nearest_group = variable_binning.loc[(variable_binning[Bad_Rate] - bad_rate).abs().idxmin()]
    elif method == "highest":
        # Find the group with the highest bad rate
        nearest_group = variable_binning.loc[variable_binning[Bad_Rate].idxmax()]
    elif method == "lowest":
        # Find the group with the lowest bad rate
        nearest_group = variable_binning.loc[variable_binning[Bad_Rate].idxmin()]
    elif method == "average":
        # Find the group closest to the average bad rate
        average_bad_rate = variable_binning[Bad_Rate].mean()
        nearest_group = variable_binning.loc[(variable_binning[Bad_Rate] - average_bad_rate).abs().idxmin()]
    
    # Ensure `nearest_group` is a Series before accessing `Group`
    return nearest_group[Group]



# ////////////////////////////////////////////////////////////////////////////////////////////////////////////

unmapped_details = []

# try:

# Process each variable in the binning dataframe
for column in tqdm(binning_df[Variable].unique(),desc="Processing Time = "):
    # Extract bins and groups for the current column
    bins = binning_df[binning_df[Variable] == column][Bin].tolist()
    groups = binning_df[binning_df[Variable] == column][Group].tolist()

    # Map values to bins
    data[f"{column}_{name}"] = [map_to_bins_categorical(value, bins, groups) for value in data[column]]

    # Compute bad rates only if target is not None
    bad_rate_df = None
    if target is not None:
        bad_rate_df = compute_bad_rate(data, column, target)

    # Handle unmapped values
    unique_values = data[column].unique()
    for value in unique_values:
        if data.loc[data[column] == value, f"{column}_{name}"].isna().all():
            bad_rate = None
            nearest_group = None

            # If target exists, calculate the nearest group based on the bad rate
            if bad_rate_df is not None:
                bad_rate_row = bad_rate_df.loc[bad_rate_df[column] == value, Bad_Rate]
                if not bad_rate_row.empty:
                    bad_rate = bad_rate_row.values[0]
                    nearest_group = find_nearest_group(bad_rate, binning_df, column, methods)
            else:
                # Use the pre-selected method if no target is available
                nearest_group = find_nearest_group(None, binning_df, column, methods)

            # Log details of unmapped values
            unmapped_details.append({
                "Column": column,
                "Unmapped_Value": value,
                "Method": methods,
                "Assigned_Bad_Rate": bad_rate,
                "Assigned_Group": nearest_group
            })

            # Map the unmapped value to the determined group
            data.loc[data[column] == value, f"{column}_{name}"] = nearest_group


            # Update the binning_df with the new bin
            existing_bin_rows = binning_df[(binning_df[Variable] == column) & (binning_df[Group] == nearest_group)]

            new_bin_value = f"[{value}]"

            if not existing_bin_rows.empty:
                # Update existing bin with the new value
                existing_bins = existing_bin_rows.iloc[0][Bin]

                # Parse the existing bin into a list (removing brackets and splitting by commas)
                existing_bin_list = existing_bins.strip("[]").split(",")

                # Ensure all elements are strings to avoid type errors
                existing_bin_list = [str(item).strip() for item in existing_bin_list if item]

                # Append the new value if it doesn't already exist
                if str(value) not in existing_bin_list:
                    existing_bin_list.append(str(value))

                    # Sort the list and reconstruct it into the exact required format
                    sorted_bin_list = sorted(existing_bin_list)
                    updated_bin_V1 = "['" + "' '".join(sorted_bin_list) + "']"
                    updated_bin_V2 = updated_bin_V1.replace("\n", "").replace("''", "'").replace(", ", " ").replace("]", "").replace("[", "['").strip() + "]"
                    updated_bin_V3 = re.sub(r"\['+'", "['", updated_bin_V2)

                    # Update only the rows where the value was added
                    binning_df.loc[existing_bin_rows.index, Bin] = updated_bin_V3
            else:
                # Add a new row if no bin exists
                updated_bin_V1 = f"['{value}']"  # Create the initial bin in the required format
                # binning_df = binning_df.append({Variable: column, Bin: updated_bin_V1}, ignore_index=True)
                binning_df = pd.concat([binning_df, pd.DataFrame([{Variable: column, Bin: updated_bin_V1}])], ignore_index=True)
# except:
#     pass
# ////////////////////////////////////////////////////////////////////////////////////////////////////////////

# Convert unmapped details to a DataFrame
unmapped_df = pd.DataFrame(unmapped_details)

# ////////////////////////////////////////////////////////////////////////////////////////////////////////////

# Display the updated DataFrame
print("Updated DataFrame:")
print(data.shape)

# Updated binning_df with new bins for unmapped values
print("\nUpdated Binning DataFrame:")
print(binning_df.shape)

# Display the unmapped values DataFrame
print("\nUnmapped Details DataFrame:")
unmapped_df.head(5)


Processing Time = :   0%|          | 0/5 [00:00<?, ?it/s]

Processing Time = : 100%|██████████| 5/5 [00:00<00:00,  5.07it/s]

Updated DataFrame:
(7768, 250)

Updated Binning DataFrame:
(16, 14)

Unmapped Details DataFrame:


,Column,Unmapped_Value,Method,Assigned_Bad_Rate,Assigned_Group
0,CUSTOMER_SEGMENT_SOBRE_NEW,NaN,highest,None,2
1,PRODUCT_CUBE_NEW,ALLT1511,highest,None,4
2,PRODUCT_CUBE_NEW,ACEGOLD+DIESEL,highest,None,4
3,PRODUCT_CUBE_NEW,MAHINDRAJEETOL7,highest,None,4
4,PRODUCT_CUBE_NEW,ALBADADOSTI2,highest,None,4


In [18]:
unmapped_df

,Column,Unmapped_Value,Method,Assigned_Bad_Rate,Assigned_Group
0,CUSTOMER_SEGMENT_SOBRE_NEW,NaN,highest,None,2
1,PRODUCT_CUBE_NEW,ALLT1511,highest,None,4
2,PRODUCT_CUBE_NEW,ACEGOLD+DIESEL,highest,None,4
3,PRODUCT_CUBE_NEW,MAHINDRAJEETOL7,highest,None,4
4,PRODUCT_CUBE_NEW,ALBADADOSTI2,highest,None,4
...,...,...,...,...,...
89,CUST_DL_PROP_ITR_V1,RTLC1_YES_YES_NO,highest,None,2
90,CUST_DL_PROP_ITR_V1,RTLC2_YES_NO_NO,highest,None,2
91,CUST_DL_PROP_ITR_V1,RTL2_NO_NO_NO,highest,None,2
92,CUST_DL_PROP_ITR_V1,RTL2_YES_YES_NO,highest,None,2


In [19]:
data.isna().sum().to_dict()

{'Ref ID': 0,
 'CONNO': 0,
 'LOGIN_DATE': 0,
 'REQUEST-TIME': 434,
 'AD_B': 1,
 'AGE_B': 0,
 'AGE_NEW_V1': 0,
 'APPLICATION_USAGE': 825,
 'ASSET_AGE_B': 649,
 'ASSET_AGE_NEW': 0,
 'UV_REP_SEGMENT': 1030,
 'AUTO_LOAN_COMPLETED': 7768,
 'AUTO_LOAN_LIVE': 7768,
 'BOW_CV_BIZ_EXP_B': 634,
 'BRANCH_NEW': 0,
 'CHASSIS_FULLY_BUILT': 687,
 'CO_APP_RELN_BOW': 1372,
 'COBORROWER_YN_node': 649,
 'COMMERCIAL_VEHI_COMPLETED': 7768,
 'CUST_PROFILE_ASSET_CATEGORY': 0,
 'CUST_SEG_NODE': 0,
 'DIST_SALES_OUTLET': 0,
 'DL_CLASS': 4822,
 'DL_YES_NO': 0,
 'DSA_DLR': 668,
 'EXISTING_CUST_YN_NODE': 5137,
 'EXISTING_FINANCIER': 5745,
 'FIN_AMOUNT': 0,
 'FTF_REPEAT_MULTIPLE': 0,
 'GENDER_NODE': 3005,
 'HOUSING_LOAN_COMPLETED': 7768,
 'HOUSING_LOAN_LIVE': 7768,
 'INCOME': 0,
 'IT_RETURNS': 7768,
 'IT_RETURNS_YES_NO': 0,
 'LTV': 0,
 'MARITAL_STATUS': 88,
 'NUM_VEHICLES': 0,
 'Num_Vehicle_New_Node': 0,
 'PRODUCT': 0,
 'PROPERT_TYPE': 638,
 'PROPERT_TYPE_New': 5535,
 'PROPERT_TYPE_YES_NO': 0,
 'CUST_DL_PROP_ITR_V1'

# Export Binned Data

In [20]:
# data.to_csv(rf"{path_main}\{names}_apply_binning V1.csv",index=False)
data.to_excel(rf"{path_main}\{names}_apply_binning V10.xlsx",index=False)

In [47]:
with pd.ExcelWriter(rf"{path_main}\{names}_biining file with nearest bad rate.xlsx",engine='xlsxwriter') as want:
    binning_df.to_excel(want,sheet_name="Categorical Binning",index=False)
    df_num_binning.to_excel(want,sheet_name="Numerical Binning",index=False)
    unmapped_df.to_excel(want,sheet_name="Unmapped Values",index=False)
# binning_df.to_excel(rf"{path_main}\{names}_biining file with highest bad rate.xlsx",index=False)